In [1]:
#Install Required Libraries

In [2]:
!pip install minsearch
!pip install -q "qdrant-client[fastembed]>=1.14.2"
!pip install google-generativeai
!pip install --upgrade jupyter ipywidgets

In [3]:
import minsearch
import json
import google.generativeai as genai
import os
import requests 

In [4]:
genai.configure(api_key="#") #--- Input your Gemini API Key
model = genai.GenerativeModel('gemini-2.5-flash')

In [5]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [11]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [13]:
def llm(prompt):
    response = model.generate_content(
        {"role": "user", "parts": [prompt]}
    )
    return response.text

In [14]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [15]:
rag('the course has already started, can I still enroll?')

"Yes, you can still join the course after the start date. Even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects."

In [16]:
## RAG with Vector Search

In [17]:
#Import Required Libraries & Connect to Qdrant

In [18]:
from qdrant_client import QdrantClient, models

In [19]:
qd_client = QdrantClient("http://localhost:6333")

In [20]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [21]:
collection_name = "zoomcamp-faq"

In [22]:
qd_client.delete_collection(collection_name=collection_name)

True

In [23]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [24]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [25]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [26]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [27]:
question = 'I just discovered the course. Can I still join it?'

In [28]:
def vector_search(question):
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [29]:
def rag_vector_search(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
rag_vector_search('I just discovered the course. Can I join now?')

vector_search is used


"Yes, you can still join the course. Even if you don't register, you are eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects."

In [31]:
#Q1. Embedding the query
#Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.

#You should get a numpy array of size 512.
#What's the minimal value in this array?

In [ ]:
!pip install fastembed
from fastembed import TextEmbedding
import tqdm as notebook_tqdm
import numpy as np

In [33]:
query = 'I just discovered the course. Can I join now?'

model_name = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_name=model_name)

EMBEDDING_DIMENSIONALITY = 512

embeddings_generator = embedding_model.embed(query)
query_list = np.array(list(embeddings_generator))
min_value = np.min(query_list)

print(f"The minimal value in the embedding array is: {min_value}")

The minimal value in the embedding array is: -0.11726373885183883


In [34]:
import numpy as np

norm = np.linalg.norm(query_list)
print(f"The L2 norm of the embedding is: {norm}")

query_list.dot(norm)

The L2 norm of the embedding is: 1.0


array([[-7.63946260e-02, -7.30555505e-02,  5.86501763e-02,
         3.92670571e-02, -1.41797115e-02, -4.68342854e-02,
         2.69862822e-02,  3.49039060e-02,  1.53419217e-03,
        -1.10407363e-02,  5.57556285e-02, -2.88360142e-02,
        -4.87239498e-02, -9.90923923e-02,  6.02579764e-02,
        -1.86105379e-02, -8.81040139e-03, -3.01994831e-02,
        -1.42541815e-02, -3.87884212e-02, -2.76125662e-02,
        -6.94957533e-03,  3.20653319e-02, -5.17838394e-03,
         8.37959894e-02, -8.87329606e-02, -7.30260625e-02,
         5.92846802e-02,  4.07805674e-02,  7.72684822e-02,
        -4.60029697e-02,  3.98426895e-02,  1.74821510e-02,
         8.70263712e-03, -3.09291054e-02,  2.20470391e-02,
         4.76479896e-02,  1.76566196e-02, -3.62013627e-02,
        -3.53408062e-02, -5.92537995e-03,  2.97091202e-02,
         8.05883752e-02,  1.07132722e-02, -5.38192519e-02,
        -9.62881111e-03, -1.17263739e-01,  3.36546374e-02,
         7.05763168e-03,  7.70159791e-03, -5.17482529e-0

In [ ]:
#Q2. Cosine similarity with another vector
#Now let's embed this document:

#doc = 'Can I still join the course after the start date?'
#What's the cosine similarity between the vector for the query and the vector for the document?

In [35]:
doc = 'Can I still join the course after the start date?'

EMBEDDING_DIMENSIONALITY = 512

doc_generator = embedding_model.embed(doc)
doc_list = np.array(list(doc_generator))
import numpy as np

# Calculate the L2 norm of the embedding
norm_doc = np.linalg.norm(doc_list)
print(f"The L2 norm of the embedding is: {norm_doc}")

query_list.dot(doc_list.T)

The L2 norm of the embedding is: 1.0


array([[0.90085289]])

In [ ]:
#Q3: Ranking by cosine
#Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.
#What's the document index with the highest similarity? (Indexing starts from 0):

In [36]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [37]:
# Extract the 'text' field from each document
document_texts = [doc["text"] for doc in documents]

print(f"\nEmbedding {len(document_texts)} document texts...")
# Embed all document texts
document_embeddings = embedding_model.embed(document_texts)


Embedding 5 document texts...


In [38]:
# Ensure embeddings are normalized for cosine similarity if not already by the model
# Jina embeddings are typically normalized, but it's good practice to ensure.

query_embedding_norm = query_list / np.linalg.norm(query_list)

In [39]:
document_list = np.array(list(document_embeddings))
document_embeddings_norm = document_list / np.linalg.norm(document_list, axis=1, keepdims=True)

In [40]:
similarities = np.dot(document_embeddings_norm, query_embedding_norm.reshape(-1, 1)).flatten()

In [41]:
print("\nCosine Similarities with Query:")
for i, sim in enumerate(similarities):
    print(f"Document {i}: {sim:.4f}")


Cosine Similarities with Query:
Document 0: 0.7630
Document 1: 0.8182
Document 2: 0.8085
Document 3: 0.7133
Document 4: 0.7304


In [42]:
# Find the index of the document with the highest similarity
highest_similarity_index = np.argmax(similarities)
max_similarity_value = similarities[highest_similarity_index]

print(f"\nDocument index with the highest similarity: {highest_similarity_index}")
print(f"Highest similarity value: {max_similarity_value:.4f}")


Document index with the highest similarity: 1
Highest similarity value: 0.8182


In [ ]:
#Q4. Ranking by cosine, version two
#Now let's calculate a new field, which is a concatenation of question and text:
#full_text = doc['question'] + ' ' + doc['text']
#Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

In [43]:
full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]
model_name = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_name=model_name)

EMBEDDING_DIMENSIONALITY = 512

full_embeddings = embedding_model.embed(full_text)
full_list = np.array(list(full_embeddings))
full_embedding_norm = full_list / np.linalg.norm(full_list, axis=1, keepdims=True)

query_embedding_norm = query_list / np.linalg.norm(query_list)
# Print shapes for verification
print(full_embedding_norm.shape)
print(query_embedding_norm.shape)

similarities = np.dot(query_embedding_norm, full_embedding_norm.T)

(5, 512)
(1, 512)


In [44]:
print("\nCosine Similarities with Query:")
# Flatten the similarities array to make it a 1D array
for i, sim_value in enumerate(similarities.flatten()):
    print(f"Document {i}: {sim_value:.4f}")

# Find the index of the document with the highest similarity
highest_similarity_index = np.argmax(similarities)
max_similarity_value = similarities[highest_similarity_index]

print(f"Highest similarity value: {max_similarity_value[0]:.4f}")


Cosine Similarities with Query:
Document 0: 0.8515
Document 1: 0.8437
Document 2: 0.8408
Document 3: 0.7755
Document 4: 0.8086
Highest similarity value: 0.8515


In [ ]:
#Q5. Selecting the embedding model
#Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

In [45]:

full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]
model_name = "BAAI/bge-small-en"
embedding_model = TextEmbedding(model_name=model_name)

EMBEDDING_DIMENSIONALITY = 384 

full_embeddings = embedding_model.embed(full_text)
full_list = np.array(list(full_embeddings))
full_embedding_norm = full_list / np.linalg.norm(full_list, axis=1, keepdims=True)

embeddings_generator = embedding_model.embed(query)
query_list = np.array(list(embeddings_generator))
min_value = np.min(query_list)
query_embedding_norm = query_list / np.linalg.norm(query_list)
# Print shapes for verification
print(full_embedding_norm.shape)
print(query_embedding_norm.shape)

similarities = np.dot(query_embedding_norm, full_embedding_norm.T)

(5, 384)
(1, 384)


In [67]:
print("\nCosine Similarities with Query:")
# Flatten the similarities array to make it a 1D array
for i, sim_value in enumerate(similarities.flatten()):
    print(f"Document {i}: {sim_value:.4f}")

# Find the index of the document with the highest similarity
highest_similarity_index = np.argmax(similarities)
max_similarity_value = similarities[highest_similarity_index]

print(f"Highest similarity value: {max_similarity_value[0]:.4f}")


Cosine Similarities with Query:
Document 0: 0.8508
Document 1: 0.8354
Document 2: 0.8469
Document 3: 0.8225
Document 4: 0.8297
Highest similarity value: 0.8508


In [47]:
#Q6. Indexing with qdrant
#We will select only FAQ records from our ml zoomcamp: 
#Add them to qdrant using the model form Q5.
#When adding the data, use both question and answer fields:
#text = doc['question'] + ' ' + doc['text']
#After the data is inserted, use the question from Q1 for querying the collection.
#What's the highest score in the results? (The score for the first returned record):

In [76]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

EMBEDDING_DIMENSIONALITY = 384 

text = [doc['question'] + ' ' + doc['text'] for doc in documents]

In [88]:
from fastembed import TextEmbedding
import numpy as np
model_name = "BAAI/bge-small-en"
embedding_model = TextEmbedding(model_name=model_name)

EMBEDDING_DIMENSIONALITY = 384 

text_embeddings = embedding_model.embed(text)
text_list = np.array(list(text_embeddings))
text_embedding_norm = text_list / np.linalg.norm(text_list, axis=1, keepdims=True)
#text_embedding_norm = text_list / np.linalg.norm(text_list)

In [98]:
query = 'I just discovered the course. Can I join now?'
embeddings_generator = embedding_model.embed(query)
query_list = np.array(list(embeddings_generator))
query_embedding_norm = query_list / np.linalg.norm(query_list)

# Print shapes for verification
print(text_embedding_norm.shape)
print(query_embedding_norm.shape)

similarities = np.dot(text_embedding_norm, query_embedding_norm.T)

(375, 384)
(1, 384)


In [99]:
print(similarities[0])

[0.8147728]


In [100]:
print("\nCosine Similarities with Query:")
# Find the index of the document with the highest similarity
highest_similarity_index = np.argmax(similarities)
max_similarity_value = similarities[highest_similarity_index]

print(f"Highest similarity value: {max_similarity_value[0]:.4f}")


Cosine Similarities with Query:
Highest similarity value: 0.8703
